In [ ]:
import sirf.STIR as stir
import sirf.Reg as reg
import numpy as np
import os
import sys
# add the source directory to the path
# This is to make everything nice and self-contained
dir_path = os.path.dirname(os.getcwd())
source_path = os.path.join(dir_path, 'source')
sys.path.append(source_path)

from reconstruction.reconstruction import *
from reconstruction.registration import *
from reconstruction.osem import *
from plotting_functions import plot_2d_image

# and some additional functions for plotting
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
import imageio
from plotting_functions import *
from IPython.display import Image

In [ ]:
# First, let's get some template data
data_path = os.path.join(dir_path, 'data', 'template_data')
emission_image = stir.ImageData(os.path.join(data_path, 'emission.hv'))
attenuation_image = stir.ImageData(os.path.join(data_path, 'attenuation.hv'))
template_sinogram = stir.AcquisitionData(os.path.join(data_path, 'template_sinogram.hs'))

In [ ]:
num_motion_states = 10
num_osem_subsets = 8

In [ ]:
motion_emissions, motion_attenuations = [transformed_images for transformed_images in zip(*[generate_transformed_image(emission_image, attenuation_image) for i in range(num_motion_states)])]

In [ ]:
acquisition_models = [get_acquisition_model(emission_image, template_sinogram, attn, num_subsets=num_osem_subsets) for attn in motion_attenuations]
acquired_data = [get_acquired_data(image, acq_model) for image, acq_model in zip(motion_emissions, acquisition_models)]
sensitvity_images = [acq_model.backward(acquired_data[i].get_uniform_copy(1)) for i, acq_model in enumerate(acquisition_models)]

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(20, 3))
for i in range(2):
    for j in range(5):
        plot_2d_image([2,5,5*i+j+1], acquired_data[5*i+j].as_array()[0,0], "data")